In [1]:
import numpy as np
from helpers import *
from data_processing import*
from cross_validation import *

In [2]:
x_train,x_test,y_train,train_ids, test_ids = load_csv_data('./dataset/dataset/dataset_to_release')


In [3]:
x_train_preprocessed = clean_all(x_train)
x_test_preprocessed = clean_all(x_test)

In [4]:
print(x_test_preprocessed.shape)
print(x_train_preprocessed.shape)

(109379, 66)
(328135, 66)


In [5]:
x_tr, x_te, y_tr, y_te = split_data(x_train_preprocessed,y_train,0.95)

[  5034 271803 120350 ... 312201 267336 128037]


In [6]:
f1_regression_score = {'GD': None,
                       'SGD': None,
                       'LS': None,
                       'Ridge': None,
                       'Log': None,
                       'RegLog': None}

best_parameters_per_regression_model = {'GD': None,
                       'SGD': None,
                       'LS': None,
                       'Ridge': None,
                       'Log': None,
                       'RegLog': None}

In [37]:
initial_ws = []
initial_ws.append('random')
k_fold = 5
max_iter = 1000
gammas = np.arange(0.001,0.02,0.001)
gammas = np.round(gammas,3)
degrees = [0]
lambdas = np.arange(0.001,0.51,0.005)
lambdas = np.round(lambdas,4)

In [40]:
best_parameters_per_regression_model['GD'] = best_selection_gd(y_tr,x_tr,np.arange(3),k_fold,initial_ws,max_iter,gammas)

KeyboardInterrupt: 

In [9]:
best_parameters_per_regression_model['SGD'] = best_selection_sgd(y_tr,x_tr,degrees,k_fold,initial_ws,max_iter,gammas)


Divergence detected. Stopping iteration.
Divergence detected. Stopping iteration.
Divergence detected. Stopping iteration.
Divergence detected. Stopping iteration.
Divergence detected. Stopping iteration.
Divergence detected. Stopping iteration.
Divergence detected. Stopping iteration.
Divergence detected. Stopping iteration.
Divergence detected. Stopping iteration.
Divergence detected. Stopping iteration.
Divergence detected. Stopping iteration.
Divergence detected. Stopping iteration.
Divergence detected. Stopping iteration.
Divergence detected. Stopping iteration.
Divergence detected. Stopping iteration.
Divergence detected. Stopping iteration.
Divergence detected. Stopping iteration.
Divergence detected. Stopping iteration.
Divergence detected. Stopping iteration.
Divergence detected. Stopping iteration.
Divergence detected. Stopping iteration.
Divergence detected. Stopping iteration.
Divergence detected. Stopping iteration.
Divergence detected. Stopping iteration.
Divergence detec

In [38]:
best_parameters_per_regression_model['Ridge'] = best_selection_ridge(y_tr,x_tr,degrees,k_fold,lambdas)


In [11]:
best_parameters_per_regression_model['Log'] = best_selection_logistic(y_tr,x_tr,degrees,k_fold,initial_ws,max_iter,gammas)


Divergence detected. Stopping iteration.
Divergence detected. Stopping iteration.
Divergence detected. Stopping iteration.
Divergence detected. Stopping iteration.
Divergence detected. Stopping iteration.
Divergence detected. Stopping iteration.
Divergence detected. Stopping iteration.
Divergence detected. Stopping iteration.
Divergence detected. Stopping iteration.
Divergence detected. Stopping iteration.
Divergence detected. Stopping iteration.
Divergence detected. Stopping iteration.
Divergence detected. Stopping iteration.
Divergence detected. Stopping iteration.
Divergence detected. Stopping iteration.
Divergence detected. Stopping iteration.
Divergence detected. Stopping iteration.
Divergence detected. Stopping iteration.
Divergence detected. Stopping iteration.
Divergence detected. Stopping iteration.


In [12]:
best_parameters_per_regression_model['RegLog'] = best_selection_reg_logistic(y_tr,x_tr,degrees,k_fold,max_iter,initial_ws,lambdas,gammas)


KeyboardInterrupt: 

In [14]:
np.random.seed(42)
ini_w = np.random.rand(x_train_preprocessed.shape[1], 1)

In [39]:
print(best_parameters_per_regression_model)

{'GD': {'degree': 0, 'gamma': 0.016, 'initial_w': 'random'}, 'SGD': {'degree': 0, 'gamma': 0.005, 'initial_w': 'random'}, 'LS': None, 'Ridge': {'degree': 0, 'lambda': 0.001}, 'Log': {'degree': 0, 'gamma': 0.015, 'initial_w': 'random'}, 'RegLog': None}


In [ ]:
w_gd, loss_gd = mean_squared_error_gd(y_train,x_train_preprocessed,ini_w,max_iter,best_parameters_per_regression_model['GD']['gamma'])
w_sgd, loss_sgd = mean_squared_error_sgd(y_train,x_train_preprocessed,ini_w,max_iter,best_parameters_per_regression_model['SGD']['gamma'])
w_ls, loss_ls = least_squares(y_train,x_train_preprocessed)
w_ridge, loss_ridge = ridge_regression(y_train,x_train_preprocessed,best_parameters_per_regression_model['Ridge']['lambda'])
w_log, loss_log = logistic_regression(y_train,x_train_preprocessed,ini_w,max_iter,best_parameters_per_regression_model['Log']['gamma'])
#w_reglog, loss_reglog = reg_logistic_regression(y_train,x_train_preprocessed,best_parameters_per_regression_model['RegLog']['lambda'],ini_w,max_iter,best_parameters_per_regression_model['RegLog']['gamma'])

In [ ]:
f1_regression_score['GD'] = compute_f1_score(y_te,prediction(x_te,w_gd))
f1_regression_score['SGD'] = compute_f1_score(y_te,prediction(x_te,w_sgd))
f1_regression_score['LS'] = compute_f1_score(y_te,prediction(x_te,w_ls))
f1_regression_score['Ridge'] = compute_f1_score(y_te,prediction(x_te,w_ridge))
f1_regression_score['Log'] = compute_f1_score(y_te,prediction(x_te,w_log))
#f1_regression_score['RegLog'] = compute_f1_score(y_te,prediction(x_te,w_reglog))

In [ ]:
print(f1_regression_score)

In [19]:
best_score = -1  
best_method = None  

for method, score in f1_regression_score.items():
    if score is not None:
        if score > best_score:  
            best_score = score  
            best_method = method  

w_pred  = 0
loss_pred = 0
if best_method == 'GD':
    w_pred = w_gd
if best_method == 'SGD':
    w_pred = w_sgd
if best_method == 'LS':
    w_pred = w_ls
if best_method == 'Ridge':
    w_pred = w_ridge
if best_method == 'Log':
    w_pred = w_log
#if best_method == 'Reglog':
    #w_pred = w_reglog


In [20]:
y_pred = prediction(x_test_preprocessed,w_pred)

In [21]:
create_csv_submission(test_ids,y_pred, 'submission')